In [1]:
import sys
import numpy as np
import pandas as pd
import dill
# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
# Matplotlibのインライン表示
%matplotlib inline

In [2]:
# データ読み込み
df = pd.read_csv('./data/equip_training.txt')
# データの重複を確認
# print(df.duplicated())
# print(df[df.duplicated()])
# データの重複を削除
print(df[~df.duplicated()].shape)
# データとラベルに分類（データの1行目にラベルデータが必要）
x = df.loc[:, ['symbol','width','height','unit','amount']]
y = df.loc[:, "ups"]

(1545, 6)


In [3]:
# 分類
# train_x, test_x, train_y, test_y = train_test_split(x, y)
# シャッフルされる場合、デフォルトでは実行するたびにランダムに分割される。引数random_seedを指定して乱数シードを固定すると常に同じように分割される。
seed = 42
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=seed)

In [4]:
# # ハイパーパラメータをグリッドサーチを用いて調査
# from sklearn.model_selection import GridSearchCV
# search_params = {
#       'n_estimators'      : [5, 10, 20, 30, 50, 100, 300],
#       'max_features'      : [3, 5, 10, 15, 20],
#       'random_state'      : [seed],
#       'n_jobs'            : [1],
#       'min_samples_split' : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100],
#       'max_depth'         : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100]
# }
# gs = GridSearchCV(RandomForestClassifier(),  # 対象の機械学習モデル
#                   search_params,             # 探索パラメタ辞書
#                   cv=3,                      # クロスバリデーションの分割数
#                   verbose=True,              # ログ表示
#                   n_jobs=-1)                 # 並列処理
# gs.fit(train_x, train_y)
 
# print(gs.best_estimator_)

In [5]:
# ハイパーパラメータチューニング
clf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                             criterion='gini', max_depth=20, max_features=5,
                             max_leaf_nodes=None, max_samples=None,
                             min_impurity_decrease=0.0, min_impurity_split=None,
                             min_samples_leaf=1, min_samples_split=3,
                             min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
                             oob_score=False, random_state=seed, verbose=0,
                             warm_start=False)

# 学習
clf.fit(train_x,train_y)
print(clf.score(test_x,test_y))
# 未知のデータ
# unknown_data = [[402,10.798,0.443,4.783]] # 02170050
# unknown_data = [[23,0,0,30,1]]      # 0271128002720200
unknown_data = [[23,0,0,30,1]]       # 0271128002720200
# 推論
print(clf.predict(unknown_data))

0.550321199143469
["'0271128002720200'"]


In [6]:
# モデルを保存する
fp = 'equip_autoup.model'
# pickle.dump(clf, open(fp, 'wb'))
dill.dump(clf, open(fp,'wb'))

In [7]:
# 学習済みモデルを復元する
# model = pickle.load(open(fp,'rb'))
# model = dill.load(open(fp,'rb'))